## API example

Что можно сделать с помощью __ml_api__?
1. Вернуть список доступных для обучения классов моделей
2. Обучить ML-модель с возможностью настройки гиперпараметров. При этом гиперпараметры для разных моделей могут быть разные. Количество классов моделей доступных для обучения = 4
3. Вернуть предсказание конкретной модели (система умеет хранить несколько обученных моделей)
4. Обучить заново и удалить уже обученные модели

С помощью __results__ можно посмотреть статус или результат задачи обучения модели или расчета предсказаний, выполняемой другим контейнером, по ее id.

С помощью __metrics__ можно выполнять CRUD операции с базой данных метрик качества обученных ML-моделей.

---

Сперва необходимо запустить утилиту `docker-compose`

In [1]:
import requests
import io
import pandas as pd
from sklearn.datasets import load_iris, load_boston

Демонстрировать возможности API будем на датасетах _boston_ (задача регрессии) и _iris_ (задача классификации)

In [2]:
iris_data = load_iris()
iris_df = pd.DataFrame(iris_data.data, columns=iris_data.feature_names)
iris_df['species'] = iris_data.target
iris_json = iris_df.to_json()

boston_data = load_boston()
boston_df = pd.DataFrame(boston_data.data, columns=boston_data.feature_names)
boston_df['PRICE'] = boston_data.target
boston_json = boston_df.to_json()

Метод __get__ возвращает список доступных для обучения моделей, а точнее алгоритмов из `sklearn`. Это сделано для того, чтобы сразу определялся тип задачи – регрессия или классификация. Всего реализовано 4 класса моделей:
- Линейные модели
- Метод опорных векторов
- Дерево решений
- Случайный лес

In [3]:
resp = requests.get('http://0.0.0.0:8080/ml_api')
resp.json()

['LogisticRegression',
 'SVC',
 'DecisionTreeClassifier',
 'RandomForestClassifier',
 'Ridge',
 'SVR',
 'DecisionTreeRegressor',
 'RandomForestRegressor']

Чтобы обучить модель, необходимо вызвать метод __post__ и передать в параметр __json__ всю необходимую информацию, а именно:
- model (str) – модель, которую необходимо обучить (нужно выбрать из списка)
- data (json) – данные в формате json, на которых надо обучить модель (поддерживаются только количественные признаки, таргет – последний элемент файла)
- params (dict) – гиперпараметры модели, по умолчанию используются дефолтные значения соответствующего алгоритма в `sklearn`
- grid_search (bool) – надо ли делать подбор оптимальных значений гиперпараметров, по умолчанию нет
- param_grid (dict) – сетка для перебора значений гиперпараметров, по умолчанию используется заданная сетка

Внутри данного метода, помимо сохранения обучающих и тестовых данных (разбиение производится автоматически случайным образом в соотношении 7:3), обучения модели и сохранения модели, также рассчитываются метрики качества в зависимости от класса модели, которые затем могут быть записаны в базу данных. Пример см ниже.

Основное отличие от первой версии приложения это то, что теперь методы, предполагающие обучение модели и расчет предсказаний, выдают id таска, результаты выполнения которого можно посмотреть __get__-запросом по пути `/results/<model_id>`.

In [4]:
payload = {'model': 'RandomForestRegressor', 'data': boston_json}
response = requests.post('http://0.0.0.0:8080/ml_api', json=payload)
response.status_code, response.json()

(200, 'Task_id = b09bf95a-516d-4ff9-bcf6-e03f4ad9ba30')

In [6]:
response = requests.get('http://0.0.0.0:8080/results/b09bf95a-516d-4ff9-bcf6-e03f4ad9ba30')
response.status_code, response.json()

(200, 'RandomForestRegressor is fitted and saved')

Чтобы посмотреть информацию обо всех обученных моделях, надо вызвать метод __get__ по адресу `/ml_api/all_models`. Для каждой модели хранится ее id, название, гиперпараметры, было ли обучение модели заново и была ли она удалена. Это своего рода файл истории использования API. Данные и модель сохраняются в файлы, в названии которых указан id модели, в папках `./worker/data` и `./worker/models` соответственно. 

In [7]:
response = requests.get('http://0.0.0.0:8080/ml_api/all_models')
response.json()

{'1': {'model': 'RandomForestRegressor',
  'params': {'bootstrap': True,
   'ccp_alpha': 0.0,
   'criterion': 'mse',
   'max_depth': None,
   'max_features': 'auto',
   'max_leaf_nodes': None,
   'max_samples': None,
   'min_impurity_decrease': 0.0,
   'min_impurity_split': None,
   'min_samples_leaf': 1,
   'min_samples_split': 2,
   'min_weight_fraction_leaf': 0.0,
   'n_estimators': 100,
   'n_jobs': None,
   'oob_score': False,
   'random_state': None,
   'verbose': 0,
   'warm_start': False},
  'retrained': False,
  'deleted': False}}

Параметры модели, также как и сетку для их перебора, можно либо задать вручную, либо не задавать вовсе. В последнем случае используются дефолтные параметры. 

In [8]:
payload = {'model': 'RandomForestRegressor',
           'params': {'n_estimators': 200, 'max_depth': 10, 'min_samples_leaf': 3},
           'data': boston_json}
response = requests.post('http://0.0.0.0:8080/ml_api', json=payload)
response.status_code, response.json()

(200, 'Task_id = 146b2cd1-bbcd-44af-b91f-adf8591714c0')

In [9]:
response = requests.get('http://0.0.0.0:8080/results/146b2cd1-bbcd-44af-b91f-adf8591714c0')
response.status_code, response.json()

(200, 'RandomForestRegressor is fitted and saved')

API поддерживает работу только с количественными признаками. Если попытаться подать категориальные данные, выпадет ошибка 400. 

In [10]:
boston_df_cat = pd.DataFrame(boston_data.data, columns=boston_data.feature_names)
boston_df_cat['category'] = 'a'
boston_df_cat['PRICE'] = boston_data.target
boston_json_cat = boston_df_cat.to_json()

In [11]:
payload = {'model': 'RandomForestRegressor', 'data': boston_json_cat}
response = requests.post('http://0.0.0.0:8080/ml_api', json=payload)
response.status_code, response.json()

(400, 'Could not support categorical features')

Также ошибка выпадет, если пытаться строить модель не из списка доступных; использовать модель классификации для решения задачи регрессии и наоборот; передавать неправильный гиперпараметр (не поддерживаемый данным алгоритмом)

In [12]:
payload = {'model': 'RandomForestClassifier', 'data': boston_json}
response = requests.post('http://0.0.0.0:8080/ml_api', json=payload)
response.status_code, response.json()

(400, 'RandomForestClassifier can only be used for classification tasks')

In [13]:
payload = {'model': 'GradientBoostingRegressor', 'data': boston_json}
response = requests.post('http://0.0.0.0:8080/ml_api', json=payload)
response.status_code, response.json()

(404,
 "Can only train one of ['LogisticRegression', 'SVC', 'DecisionTreeClassifier', 'RandomForestClassifier', 'Ridge', 'SVR', 'DecisionTreeRegressor', 'RandomForestRegressor'] models")

In [14]:
payload = {'model': 'RandomForestRegressor', 'params': {'learning_rate': 0.1}, 'data': boston_json}
response = requests.post('http://0.0.0.0:8080/ml_api', json=payload)
response.status_code, response.json()

(400,
 "RandomForestRegressor got an unexpected keyword argument 'learning_rate'")

Вернуть предсказания конкретной модели на трейне и тесте можно с помощью метода __get__ с указанием в пути id модели – сначала вернется task id, по которому затем можно посмотреть сами предсказания. Например:

In [15]:
response = requests.get('http://0.0.0.0:8080/ml_api/1')
response.json()

'Task_id = 61a85e9b-8f92-4572-931f-a24725f119f2'

In [16]:
response = requests.get('http://0.0.0.0:8080/results/61a85e9b-8f92-4572-931f-a24725f119f2')
response.json()

{'train_predictions': '[27.08, 20.94, 19.4, 22.51, 18.22, 24.33, 33.73, 6.89, 26.43, 19.09, 21.23, 23.15, 22.36, 21.76, 47.33, 15.53, 17.25, 22.37, 20.42, 20.07, 21.0, 35.84, 22.26, 20.07, 19.87, 28.51, 36.19, 24.54, 12.89, 12.96, 11.64, 16.76, 32.51, 27.22, 13.89, 13.94, 45.13, 21.37, 20.45, 23.07, 18.62, 12.15, 6.8, 30.78, 26.2, 19.19, 15.38, 14.04, 22.99, 17.58, 27.13, 35.14, 23.03, 24.16, 23.66, 49.29, 33.84, 31.41, 23.87, 22.59, 14.89, 42.25, 19.56, 31.47, 25.42, 20.97, 21.6, 9.22, 45.45, 42.76, 31.94, 9.57, 16.69, 20.32, 34.09, 17.17, 42.28, 21.38, 23.08, 12.35, 19.85, 23.69, 29.67, 28.68, 21.95, 29.45, 24.98, 32.82, 24.4, 20.97, 24.22, 10.24, 23.83, 22.91, 16.41, 9.03, 19.71, 24.6, 23.34, 19.91, 22.55, 13.24, 30.18, 26.59, 33.89, 13.65, 15.57, 13.19, 14.08, 11.2, 24.48, 16.74, 11.26, 21.85, 14.63, 24.38, 32.45, 40.11, 29.88, 19.63, 30.46, 43.71, 23.74, 20.94, 23.44, 18.69, 18.43, 7.33, 20.61, 20.07, 27.85, 27.99, 20.52, 22.94, 14.92, 13.04, 35.97, 18.05, 15.17, 22.43, 22.28, 20.

Под обучением заново понимается обучение модели на новых данных. Сделать это можно с помощью метода __put__. Необходимо в адресе запроса указать id модели, а в параметр __json__ передать новые данные (также в формате json). С помощью полученного task id можно убедиться, что модель была переобучена. В этом случае в словаре моделей ключ retrained примет значение True для данной модели. 

In [17]:
response = requests.put('http://0.0.0.0:8080/ml_api/1', json=boston_df.sample(100).to_json())
response.json()

'Task_id = db6de01c-71ca-44f3-be82-3190488af4c0'

In [18]:
response = requests.get('http://0.0.0.0:8080/results/db6de01c-71ca-44f3-be82-3190488af4c0')
response.json()

'Model 1 is re-fitted and saved'

In [19]:
response = requests.get('http://0.0.0.0:8080/ml_api/all_models')
response.json()['1']

{'model': 'RandomForestRegressor',
 'params': {'bootstrap': True,
  'ccp_alpha': 0.0,
  'criterion': 'mse',
  'max_depth': None,
  'max_features': 'auto',
  'max_leaf_nodes': None,
  'max_samples': None,
  'min_impurity_decrease': 0.0,
  'min_impurity_split': None,
  'min_samples_leaf': 1,
  'min_samples_split': 2,
  'min_weight_fraction_leaf': 0.0,
  'n_estimators': 100,
  'n_jobs': None,
  'oob_score': False,
  'random_state': None,
  'verbose': 0,
  'warm_start': False},
 'retrained': True,
 'deleted': False}

Чтобы удалить уже обученную модель, надо вызвать метод __delete__ и указать в адресе id модели. При корректном выполнении выпадает ошибка 204, а deleted в словаре меняет свое значение на True.

In [20]:
response = requests.delete('http://0.0.0.0:8080/ml_api/1')
response

<Response [204]>

In [22]:
response = requests.get('http://0.0.0.0:8080/ml_api/all_models')
response.json()['1']

{'model': 'RandomForestRegressor',
 'params': {'bootstrap': True,
  'ccp_alpha': 0.0,
  'criterion': 'mse',
  'max_depth': None,
  'max_features': 'auto',
  'max_leaf_nodes': None,
  'max_samples': None,
  'min_impurity_decrease': 0.0,
  'min_impurity_split': None,
  'min_samples_leaf': 1,
  'min_samples_split': 2,
  'min_weight_fraction_leaf': 0.0,
  'n_estimators': 100,
  'n_jobs': None,
  'oob_score': False,
  'random_state': None,
  'verbose': 0,
  'warm_start': False},
 'retrained': True,
 'deleted': True}

При попытке вызвать какой-либо метод для удаленной модели выпадает ошибка 404. Например:

In [23]:
response = requests.get('http://0.0.0.0:8080/ml_api/1')
response.status_code, response.json()

(404, 'Model 1 was deleted')

## Database
База данных была добавлена в приложение для сохранения информации о метриках качества обучаемых моделей на трейне и тесте. Для работы с БД добавлен путь `/metrics`. Посредством post, get, put, delete запросов можно реализовать операции create, read, update, delete соответственно. 

Так, чтобы добавить записи в БД, надо вызвать __post__ и указать в адресе id уже обученной модели.

In [31]:
response = requests.post('http://0.0.0.0:8080/metrics/1')
response.status_code, response.json()

(200, 'Metrics for model 1 are added')

In [32]:
response = requests.post('http://0.0.0.0:8080/metrics/2')
response.status_code, response.json()

(200, 'Metrics for model 2 are added')

Чтобы прочитать данные из БД, надо сделать __get__ запрос. Можно посмотреть либо все записи, либо для конкретной модели.

In [33]:
response = requests.get('http://0.0.0.0:8080/metrics')
response.json()

[{'model_id': 1, 'data': 'train', 'metric': 'rmse', 'value': 1.6232},
 {'model_id': 1, 'data': 'train', 'metric': 'mae', 'value': 1.0304},
 {'model_id': 1, 'data': 'test', 'metric': 'rmse', 'value': 3.6015},
 {'model_id': 1, 'data': 'test', 'metric': 'mae', 'value': 2.3437},
 {'model_id': 2, 'data': 'train', 'metric': 'rmse', 'value': 2.2411},
 {'model_id': 2, 'data': 'train', 'metric': 'mae', 'value': 1.3764},
 {'model_id': 2, 'data': 'test', 'metric': 'rmse', 'value': 3.4637},
 {'model_id': 2, 'data': 'test', 'metric': 'mae', 'value': 2.1794}]

In [34]:
response = requests.get('http://0.0.0.0:8080/metrics/2')
response.json()

[{'data': 'train', 'metric': 'rmse', 'value': '2.2411'},
 {'data': 'train', 'metric': 'mae', 'value': '1.3764'},
 {'data': 'test', 'metric': 'rmse', 'value': '3.4637'},
 {'data': 'test', 'metric': 'mae', 'value': '2.1794'}]

Чтобы изменить данные в БД, надо сделать __put__ запрос с указанием id модели. Обновление записей в БД видится целесообразным делать после переобучения модели. Например:

In [35]:
response = requests.put('http://0.0.0.0:8080/ml_api/2', json=boston_df.sample(300).to_json())
response.json()

'Task_id = 5d39ab0e-d919-458e-99e5-942a588d7774'

In [36]:
response = requests.get('http://0.0.0.0:8080/results/5d39ab0e-d919-458e-99e5-942a588d7774')
response.json()

'Model 2 is re-fitted and saved'

In [37]:
response = requests.put('http://0.0.0.0:8080/metrics/2')
response.json()

'Metrics for model 2 are updated'

In [39]:
response = requests.get('http://0.0.0.0:8080/metrics')
response.json()

[{'model_id': 1, 'data': 'train', 'metric': 'rmse', 'value': 1.6232},
 {'model_id': 1, 'data': 'train', 'metric': 'mae', 'value': 1.0304},
 {'model_id': 1, 'data': 'test', 'metric': 'rmse', 'value': 3.6015},
 {'model_id': 1, 'data': 'test', 'metric': 'mae', 'value': 2.3437},
 {'model_id': 2, 'data': 'train', 'metric': 'rmse', 'value': 2.1444},
 {'model_id': 2, 'data': 'train', 'metric': 'mae', 'value': 1.2924},
 {'model_id': 2, 'data': 'test', 'metric': 'rmse', 'value': 3.0642},
 {'model_id': 2, 'data': 'test', 'metric': 'mae', 'value': 1.6532}]

Наконец, чтобы удалить метрики конкретной модели, надо сделать __delete__ запрос.

In [40]:
response = requests.delete('http://0.0.0.0:8080/metrics/1')
response.json()

'Metrics for model 1 are deleted'

Проверим, что записи действительно удалились из БД:

In [41]:
response = requests.get('http://0.0.0.0:8080/metrics')
response.json()

[{'model_id': 2, 'data': 'train', 'metric': 'rmse', 'value': 2.1444},
 {'model_id': 2, 'data': 'train', 'metric': 'mae', 'value': 1.2924},
 {'model_id': 2, 'data': 'test', 'metric': 'rmse', 'value': 3.0642},
 {'model_id': 2, 'data': 'test', 'metric': 'mae', 'value': 1.6532}]

Все работает! Ура!))